<a href="https://colab.research.google.com/github/kscale/QuickLearn/blob/master/makemore_part2_mlp_unoffical01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [24]:
! wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2022-10-14 06:23:30--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.009s  

2022-10-14 06:23:30 (23.5 MB/s) - ‘names.txt’ saved [228145/228145]



In [26]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [ ]:
len(words)

32033

In [28]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [32]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [33]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [34]:
# C = torch.randn((27, 2))  #将27个字符嵌入到2维空间中
C = torch.randn((27,2))

In [36]:
C

tensor([[-2.3262, -1.1028],
        [ 1.0982,  0.7169],
        [ 1.1732, -0.4150],
        [-0.5143,  0.9377],
        [ 1.0497, -1.3157],
        [-0.8922, -0.8525],
        [-0.3742, -0.7513],
        [-0.5534,  1.1095],
        [ 0.2132, -0.1377],
        [-0.4482,  0.1132],
        [ 1.0370,  0.6264],
        [ 0.1370, -1.2306],
        [-0.0703, -0.4997],
        [ 0.6661,  1.3109],
        [-1.4107, -1.7128],
        [-0.5947, -1.5274],
        [ 2.6262, -0.8022],
        [-1.7372,  0.1541],
        [ 1.0633, -1.7652],
        [ 0.8245,  0.5270],
        [ 0.4596, -1.7106],
        [ 0.6021,  0.0695],
        [ 0.5943, -1.1664],
        [ 0.2331,  1.4745],
        [ 0.4811, -0.4941],
        [ 0.3977,  0.5767],
        [ 0.6259, -0.6556]])

In [ ]:
#索引
C[5]  #5号字符放的位置 #itos[5] --->e  ??如何根据值查找得到索引

tensor([0.1136, 2.5688])

In [71]:
#X[3] #tensor([ 5, 13, 13])
#5所表示的字符在C中索引的位置
#C[3]
C[[5, 13, 13]]   #即emm 在空间中的表示

tensor([[-0.8922, -0.8525],
        [ 0.6661,  1.3109],
        [ 0.6661,  1.3109]])

In [59]:
Y.shape

torch.Size([32])

In [62]:
emb = C[X]
emb.shape
#emb

torch.Size([32, 3, 2])

把某一次中的一个输入 emm ,block_size=3
3个字符，每个字符在二维空间中的表示，即3*2=6，所以在这一个输入里，w=6

```

tensor([[-0.8922, -0.8525],
    [ 0.6661,  1.3109],
    [ 0.6661,  1.3109]]) 

```
---->



In [38]:
W1 = torch.randn((6, 100))  #为什么是6, 一次传入3个字符，2维空间 3*2，实际每次输入了6个参数
b1 = torch.randn(100)

In [39]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)   #将第一层的输出结果放到激活函数中  x*w+b

In [40]:
h

tensor([[ 1.0000, -0.9999,  0.3605,  ...,  0.3594,  1.0000, -0.9999],
        [ 1.0000, -0.9886,  0.9592,  ...,  0.9073,  0.9999, -0.9945],
        [ 0.3034,  0.8659,  0.5509,  ...,  0.9993, -0.7621, -0.1902],
        ...,
        [ 0.9239, -0.9990, -0.9999,  ...,  0.9999,  0.1563, -0.9810],
        [ 0.3496,  0.8003, -0.8885,  ..., -0.8529, -0.4737, -0.9994],
        [-0.9848,  0.9988,  0.7337,  ...,  0.4196, -0.9914,  0.2657]])

In [41]:
h.shape

torch.Size([32, 100])

In [42]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [43]:
logits = h @ W2 + b2

In [44]:
logits.shape

torch.Size([32, 27])

In [45]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)

In [46]:
prob.shape

torch.Size([32, 27])

In [60]:
prob[torch.arange(32),Y]  #用了迭代


tensor([1.0477e-16, 4.6469e-08, 1.4714e-10, 3.8296e-09, 3.0641e-09, 1.2001e-14,
        1.2487e-08, 4.1728e-10, 2.4290e-04, 1.2532e-09, 4.3890e-08, 1.0816e-08,
        5.8885e-08, 2.3810e-06, 8.7863e-13, 6.0830e-10, 4.7910e-10, 9.0093e-01,
        1.0983e-12, 6.0453e-11, 2.7012e-05, 1.2914e-13, 8.6581e-04, 2.1459e-09,
        2.2082e-07, 5.2687e-08, 4.8161e-12, 5.4566e-08, 8.4353e-16, 1.1757e-14,
        1.7531e-10, 6.1576e-08])

In [61]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(20.3247)